<a href="https://colab.research.google.com/github/GreyPaldin/CV_LB3/blob/main/LB3_vizhen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python opencv-python-headless
!git clone https://github.com/GreyPaldin/CV_LB3.git

import cv2
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab.patches import cv2_imshow
from scipy.ndimage import label
from collections import deque
import math


Cloning into 'CV_LB3'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 35 (delta 9), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 52.31 MiB | 20.89 MiB/s, done.
Resolving deltas: 100% (9/9), done.


Обнаружить объект на однородном фоне, вести его по видео. Можно рисовать траекторию движения объекта каждый кадр (доп фича).

In [ ]:
#@title Надо пробовать на нормальном видео (прототип (не актуальный))

!pip install opencv-python
import cv2
import numpy as np
from google.colab import files
from collections import deque

def rgb_to_grayscale(img):
    """Собственная реализация RGB to grayscale"""
    return (img[:,:,0]*0.299 + img[:,:,1]*0.587 + img[:,:,2]*0.114).astype(np.uint8)

def frame_diff(prev, curr, threshold=25):
    """Разница кадров с пороговой обработкой"""
    diff = np.abs(prev.astype(int) - curr.astype(int))
    return (diff > threshold).astype(np.uint8) * 255

def process_video(input_path):
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Выходные видео
    trajectory_writer = cv2.VideoWriter('trajectory.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    object_writer = cv2.VideoWriter('object.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    prev_gray = None
    trajectory = deque(maxlen=60)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Собственное преобразование в grayscale
        gray = rgb_to_grayscale(frame)

        # Всегда инициализируем object_frame
        object_frame = np.zeros_like(frame)

        if prev_gray is not None:
            # Разница кадров (собственная реализация)
            diff = frame_diff(prev_gray, gray)

            # Замена морфологических операций на OpenCV
            kernel = np.ones((5,5), np.uint8)
            closed = cv2.morphologyEx(diff, cv2.MORPH_CLOSE, kernel)

            # Замена поиска связных компонент на OpenCV
            contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if contours:
                # Используем OpenCV для выпуклой оболочки
                largest = max(contours, key=cv2.contourArea)
                hull = cv2.convexHull(largest)

                # Создаем маску объекта
                full_mask = np.zeros_like(closed)
                cv2.drawContours(full_mask, [hull], -1, 255, -1)

                # Копируем объект на черный фон
                object_frame[full_mask == 255] = frame[full_mask == 255]

                # Вычисляем центр масс
                M = cv2.moments(largest)
                if M["m00"] > 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    trajectory.append((cx, cy))

                    # Рисуем траекторию
                    for i in range(1, len(trajectory)):
                        cv2.line(frame, trajectory[i-1], trajectory[i], (255,0,0), 2)

                    # Рисуем контур объекта
                    cv2.drawContours(frame, [hull], -1, (0,255,0), 2)

        # Записываем кадры
        object_writer.write(object_frame)
        trajectory_writer.write(frame)
        prev_gray = gray.copy()

    cap.release()
    trajectory_writer.release()
    object_writer.release()

# Загружаем видео
input_filename = '/content/drive/MyDrive/Мышка2.mp4'
process_video(input_filename)

# Скачивание результатов
files.download('trajectory.mp4')
files.download('object.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Метод для видео 1
!pip install opencv-python
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
from google.colab import files
from collections import deque
import math
from scipy.ndimage import label

def rgb_to_grayscale(img):
    return (img[:,:,0]*0.299 + img[:,:,1]*0.587 + img[:,:,2]*0.114).astype(np.uint8)

def frame_diff(prev, curr, threshold=25):
    diff = np.abs(prev.astype(int) - curr.astype(int))
    return (diff > threshold).astype(np.uint8) * 255

def process_video(input_path,
                 max_line_length=100,
                 min_object_area=200,
                 max_object_area=5000,
                 detection_region_y=None):

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Ошибка открытия видеофайла")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    trajectory_writer = cv2.VideoWriter('trajectory.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    object_writer = cv2.VideoWriter('object.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    prev_gray = None
    full_trajectory = []  # Хранит все точки траектории
    current_segment = []   # Текущий непрерывный сегмент

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.GaussianBlur(rgb_to_grayscale(frame), (5,5), 0)

        object_frame = np.zeros_like(frame)

        if prev_gray is not None:
            diff = frame_diff(prev_gray, gray)
            #cv2_imshow(diff)

            # Ограничиваем область детекции
            if detection_region_y is not None:
                mask = np.zeros_like(diff)
                mask[detection_region_y:, :] = 255
                diff = cv2.bitwise_and(diff, mask)

            kernel = np.ones((5,5), np.uint8)
            closed = cv2.morphologyEx(diff, cv2.MORPH_CLOSE, kernel)
            contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if contours:
                # Фильтрация контуров по площади
                valid_contours = []
                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    if min_object_area <= area <= max_object_area:
                        valid_contours.append(cnt)

                if valid_contours:
                    largest = max(valid_contours, key=cv2.contourArea)
                    hull = cv2.convexHull(largest)
                    full_mask = np.zeros_like(closed)
                    cv2.drawContours(full_mask, [hull], -1, 255, -1)

                    # Копируем объект на черный фон
                    object_frame[full_mask == 255] = frame[full_mask == 255]

                    # Вычисляем момент и центр масс
                    M = cv2.moments(largest)
                    if M["m00"] > 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        center = (cx, cy)

                        # Обновляем траекторию
                        if current_segment:
                            last_x, last_y = current_segment[-1]
                            distance = math.sqrt((cx - last_x)**2 + (cy - last_y)**2)

                            if distance <= max_line_length:
                                current_segment.append(center)
                            else:
                                # Сохраняем текущий сегмент и начинаем новый
                                if len(current_segment) > 1:
                                    full_trajectory.append(current_segment)
                                current_segment = [center]
                        else:
                            current_segment.append(center)

                        # Рисуем контур объекта
                        cv2.drawContours(frame, [hull], -1, (0,255,0), 2)

                        # Рисуем центр маленьким квадратом
                        cv2.rectangle(frame, (cx-3, cy-3), (cx+3, cy+3), (0,0,255), -1)

        # Рисуем всю историю траектории
        for segment in full_trajectory:
            for i in range(1, len(segment)):
                cv2.line(frame, segment[i-1], segment[i], (255,0,0), 2)

        # Рисуем текущий сегмент
        if len(current_segment) > 1:
            for i in range(1, len(current_segment)):
                cv2.line(frame, current_segment[i-1], current_segment[i], (255,0,0), 2)

        object_writer.write(object_frame)
        trajectory_writer.write(frame)
        prev_gray = gray.copy()

    # Сохраняем последний сегмент
    if len(current_segment) > 1:
        full_trajectory.append(current_segment)

    cap.release()
    trajectory_writer.release()
    object_writer.release()

# Пример использования
input_filename = '/content/CV_LB3/Мышка.mp4'
process_video(input_filename,
             max_line_length=150,
             min_object_area=10000,
             max_object_area=60000,
             detection_region_y=200)
"""
process_video(input_filename,
             max_line_length=150,
             min_object_area=3000,
             max_object_area=14000,
             detection_region_y=200)
"""
files.download('trajectory.mp4')
files.download('object.mp4')

In [ ]:
#@title Метод для видео 2
def labels_pro(labels, t):
    d = {}
    for i, row in enumerate(labels[0]):
      for j, pix in enumerate(row):
        if pix not in d:
          d[pix] = [i, i, j, j, 1]
        else:
          cur = d[pix]
          cur[4] += 1
          if i < cur[0]:
            cur[0] = i
          if i > cur[1]:
            cur[1] = i
          if j < cur[2]:
            cur[2] = j
          if j > cur[3]:
            cur[3] = j


    return {k: v for k, v in d.items() if v[4] > t}

def rgb_to_grayscale(img):
    return (img[:,:,0]*0.299 + img[:,:,1]*0.587 + img[:,:,2]*0.114).astype(np.uint8)

def distance(a, b):
  return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def frame_diff(prev, curr, threshold=25):
    diff = np.abs(prev.astype(int) - curr.astype(int))
    return (diff > threshold).astype(np.uint8) * 255

def process_video(input_path,
                 max_line_length=200,
                 min_object_area=200,
                 max_object_area=5000,
                 detection_region_y=None):

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Ошибка открытия видеофайла")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    trajectory_writer = cv2.VideoWriter('trajectory.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    object_writer = cv2.VideoWriter('object.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    prev_gray = None
    full_trajectory = []  # Хранит все точки траектории
    boxes = []   # Текущий непрерывный сегмент

    last_watched_anime = None
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.GaussianBlur(rgb_to_grayscale(frame), (5,5), 0)
        gray = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]

        #cv2_imshow(gray)

        labeled_image = label(gray)
        num_labels = labeled_image[1]

        cv2_imshow(labeled_image[0])

        #anime_girls = labels_pro(labeled_image, 25000)
        anime_girls = labels_pro(labeled_image, 18000)

        #print(anime_girls)

        object_frame = np.zeros_like(frame)

        if last_watched_anime is not None:
          for k, v in last_watched_anime.items():
            center = (int((v[2] + v[3]) / 2),int((v[0] + v[1]) / 2))
            least_distant_point = None
            for k1, v1 in anime_girls.items():
              center1 = (int((v1[2] + v1[3]) / 2),int((v1[0] + v1[1]) / 2))
              if least_distant_point is None:
                least_distant_point = center1
                continue
              if distance(center, center1) < distance(least_distant_point, center):
                least_distant_point = center1
          #cv2.rectangle(frame, (v[1], v[2]), (v[3], v[0]), (0, 0, 255), 2)
          if distance(center, least_distant_point) < 600:
            full_trajectory.append([center, least_distant_point])
        last_watched_anime = anime_girls

        # Рисуем всю историю траектории
        for segment in full_trajectory:
          #print(segment[0])
          #print(segment[1])
          cv2.line(frame, segment[0], segment[1], (255,0,0), 2)


        object_writer.write(object_frame)
        trajectory_writer.write(frame)
        prev_gray = gray.copy()


    cap.release()
    trajectory_writer.release()
    object_writer.release()

# Пример использования
input_filename = '/content/CV_LB3/Мышка2.mp4'
process_video(input_filename,
             max_line_length=150,
             min_object_area=3000,
             max_object_area=14000,
             detection_region_y=200)

files.download('trajectory.mp4')